In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

#ee.Authenticate()
ee.Initialize()

### Imports and Constants
##### test areas, data sources, composite RGB image

In [181]:
import ee
import geemap

Map = geemap.Map()

total = ee.Geometry.Polygon(
        [[[25.8, -10.5],
          [25.8, -10.8],
          [26.1, -10.8],
          [26.1, -10.5]]], None, False)
area1 = ee.Geometry.Polygon(
        [[[25.75, -10.75],
          [25.75, -10.8],
          [25.9, -10.8],
          [25.9, -10.75]]], None, False)
area2 = ee.Geometry.Polygon(
        [[[25.85, -10.6],
          [25.85, -10.7],
          [25.95, -10.7],
          [25.95, -10.6]]], None, False)
area3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]], None, False)
area4 = ee.Geometry.Polygon(
        [[[25.95, -10.6],
          [25.95, -10.7],
          [26.05, -10.7],
          [26.05, -10.6]]], None, False)
noMine = ee.Geometry.Polygon(
        [[[26.05, -10.7],
          [26.05, -10.8],
          [26.15, -10.8],
          [26.15, -10.7]]], None, False)
rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]], None, False)
# Ray Areas 
roi1 = ee.Geometry.Polygon(
        [[[29.55, 3.16],
          [29.55, 3.1],
          [29.63, 3.1],
          [29.63, 3.16]]])
roi2 = ee.Geometry.Polygon(
        [[[29.76, 3.16],
          [29.76, 3.12],
          [29.8, 3.12],
          [29.8, 3.16]]])
roi3 = ee.Geometry.Polygon(
        [[[30.25, 1.8],
          [30.25, 1.7],
          [30.35, 1.7],
          [30.35, 1.8]]])

s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

### Select Area and Year

In [189]:
# input desired region of interest
area = area2
# input desired year of interest
year = 2020

### Create Composite Images (Sentinel-1 and Sentinel-2)

In [190]:
comp2020 = s2 \
            .filter(ee.Filter.bounds(area)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(area)
sar2020 = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', area)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .select(['VV', 'VH']).median().clip(area)

### Create .005x.005 Regions

In [191]:
# must be rectangular geometry
def divide_geometry(geo):
    regions = []
    coords = geo.coordinates().getInfo()[0]
    i = coords[0][0]
    while i <= coords[1][0]:
        j = coords[2][1]
        while j >= coords[0][1]:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-.005],
                  [i+.005, j-.005],
                  [i+.005, j]]])
            regions.append((region))
            j = j-.005
        i = i+.005
    return regions

In [192]:
regions = divide_geometry(area)
print('Number of Regions Created: ' + str(len(regions)))

Number of Regions Created: 420


### Create Indices: NDVI, NDMI, NIR/SWIR2

##### NDVI: Normalized Difference Vegetation Index
##### Using Modis NDVI band, values range from -2000 to 10000

In [193]:
def extract_ndvi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDVI': stats.get('NDVI')
    }
    
    return ee.Feature(None, properties)

In [194]:
def mining_modis(regions):
    Map.centerObject(area, 13)
    Map.addLayer(comp2020,rgbVis,'whole',False)
    passed = []
    for region in regions:
        image = s2 \
            .filter(ee.Filter.bounds(region)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(region)
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        dataNDVI = ee.FeatureCollection(extract_ndvi_modis(ndvi))
        dfNDVI = geemap.ee_to_pandas(dataNDVI)

        if dfNDVI.iat[0,0] < 0.4:
            passed.append(region)
    if len(passed) > 0:
        layer = ee.Geometry.MultiPolygon(passed)
        image = s2 \
            .filter(ee.Filter.bounds(layer)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer)
        Map.addLayer(image, rgbVis, 'pass')
        
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        nirG = image.normalizedDifference(['B8', 'B3'])
        ndmi = image.normalizedDifference(['B8', 'B11'])
        #iron = image.normalizedDifference(['B4', 'B2']).rename('iron')
        vv2020 = sar2020.select('VV')
        lowNDVI = ndvi.lt(0.12).rename('ndvi').selfMask()
        connectN = lowNDVI.connectedPixelCount(100)
        lowNDVI = lowNDVI.updateMask(connectN.gt(8))
        Map.addLayer(lowNDVI, {'min':0, 'max':1, 'palette':['purple']}, 'Mining')
        
        waterStorage = vv2020.lt(-15).rename('water').And(nirG.lt(0.1).Or(ndmi.gt(.3))).selfMask()
        connectW = waterStorage.connectedPixelCount(50)
        waterStorage = waterStorage.updateMask(connectW.gt(8))
        Map.addLayer(waterStorage, {'min':0, 'max':1, 'palette':['blue']}, 'Water')
                    
#         ironSpots = iron.gt(0.4).rename('iron').selfMask()
#         connectI = ironSpots.connectedPixelCount(25)
#         ironSpots = ironSpots.updateMask(connectI.gt(8))
#         Map.addLayer(ironSpots, {'min':0, 'max':1, 'palette':['gray']}, 'Iron')
                    
    return Map

In [195]:
mining_modis(regions)

Map(bottom=1111263.3999786377, center=[-10.649998588349007, 25.900000000000425], controls=(WidgetControl(optio…